In [ ]:
import gym, torch
from torch import nn
import numpy as np
import homework.hw1.load_policy as load_policy
import homework.hw1.tf_util as tf_util
import sys
import tensorflow as tf
import time


sigma = 0.1

dim = 17
dim_act = 6
obs_min = np.array([-0.25,-0.8 ,-0.7 ,-0.7,-0.65,-0.75,-0.95,-0.6 ,-1.5,-3.1,-7.1,-20,-24,-27,-27,-30,-20])
obs_max = np.array([ 0.4 , 1.65, 0.95, 0.9, 0.95, 0.95, 1.1 , 0.75, 8  , 3.4, 7  , 19, 25, 22, 25, 32, 26])

act_norm_init = np.array([2.3,2.1, 1.85,1.8 , 1.5,2.0, 2.1,2.65, 2.3,2.5, 1.9,2.3])

In [ ]:
"""

INITIALISE ACTION AND OBSERVATION SPACE

"""

actions = []
actions_matrix = np.load('HC_actions.npy')
N_a = 8
for i in range(N_a):
    actions.append(actions_matrix[i,:])

In [ ]:
"""

NEEDED FUNCTIONS

"""

''' u(x), which evaluates the function u at x=(pos,speed) with the currently set xi '''
def u(x):
    x_transformed = np.zeros(dim)
    for i in range(dim):
        x_transformed[i] = ((x[i]-obs_min[i])/(obs_max[i]-obs_min[i])-1/2)*2
    value = model_list[it](torch.from_numpy(x_transformed).float())
    return value
    
def policy(obs):
    return policy_fn(obs[None,:])
print('loading and building expert policy')
policy_fn = load_policy.load_policy('/Users/torbensell/Dropbox/UNI/CAM/PhD/Programme/BIRL_BNN_pCN/homework/hw1/experts/HalfCheetah-v2.pkl')
print('loaded and built')

''' Progress bar to know how much longer one has to wait '''
def progressBar(value, endvalue, bar_length=40):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush() 
    
def discrete_action(action_vector):
    if np.random.uniform()<0.01:
        ''' With 1% probability, return a random action '''
        rand = np.random.randint(0,N_a)
        return rand,actions[rand]
    else:
        distances = np.zeros(N_a)
        for i in range(N_a):
            distances[i] = np.linalg.norm(action_vector-actions[i])
        action = np.argmin(distances) # index of action
        return action,actions[action]
    
def discrete_action_no_noise(action_vector):
    distances = np.zeros(N_a)
    for i in range(N_a):
        distances[i] = np.linalg.norm(action_vector-actions[i])
    action = np.argmin(distances) # index of action
    return action,actions[action]

In [ ]:
"""

TEST 1 (pCN) - check if policy learned behaviour 

"""  

distances = [[],[]] # stores distances

""" initialise NNs - pCN """

method = 'pCN'
N_data = 100

model_list = []

# Hyperparameters for our network
input_size = dim
n_layer = 3
hidden_sizes = [10,10,10]
output_size = 1

# Store hyperparameters in string
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])

for it in range(1000):
    # Build a feed-forward network
    model_list.append(nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[-1], output_size)))

    # Get weights
    weights = []
    biases = []
    for l in range(n_layer+1):
        weights.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_w'+str(l)+'_sampleNo'+str(it)+'.npy'))
        biases.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_b'+str(l)+'_sampleNo'+str(it)+'.npy'))
        
    # Set all weights to 0
    for l in range(n_layer+1):
        model_list[it][2*l].weight = torch.nn.Parameter(torch.from_numpy(weights[l]).float(), requires_grad=False)
        model_list[it][2*l].bias = torch.nn.Parameter(torch.from_numpy(biases[l]).float(), requires_grad=False)

        
""" run tests - pCN """ 
        
v_with_noise = np.zeros(N_a)

print('Test started: ' + str(time.ctime()))
with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env.unwrapped
    
    for _ in range(10):
        for N_data in [100]:
            obs = env.reset()
            errors = 0  # to keep track of how many moves would be the same if using true policy
            corrects = 0

            ''' Store values taken to reproduce behaviour '''
            poss = []
            vels = []
            for k in range(100):
                progressBar(k+1,100)

                poss.append(obs[0:8])
                vels.append(obs[8:17])
                poss[k] = np.insert(poss[k],0, env.sim.data.qpos[0] )

                ''' See where different actions would take us, evaluate mean of value function samples there '''
                v = np.zeros(N_a)
                for it in range(1000): # Iterate through different neural networks
                    for i in range(N_a):
                        x = env.step(actions[i])[0]
                        v[i] += u(x).detach().numpy()[0]
                        env.set_state(poss[k], vels[k])
                v = v/1000 # to get mean of value function evaluations

                ''' Pick action which maximises the mean value function (plus noise) at the new location '''
                v_with_noise = v+sigma*np.random.normal(np.zeros(N_a),np.ones(N_a)) 
                a_argmax_with_noise = np.argmax(v_with_noise)
                
                ''' Check if action was optimal according to true policy; move according to learned behaviour  '''
                if a_argmax_with_noise != discrete_action_no_noise(policy(obs))[0]:
                    errors+=1
                else:
                    corrects+=1
                obs = env.step(actions[a_argmax_with_noise])[0] 

            print('\nErrorrate = ',(errors/(errors+corrects)))
#             input("Press Enter to start visualisation...")

            ''' Visualise the cheetah '''
            env.set_state(poss[0], vels[0])
            start = env.sim.data.qpos[0]       
            for k in range(100):
#                 env.render()
                env.set_state(poss[k], vels[k])

            print('Distance covered by Cheetah = ' + str(env.sim.data.qpos[0]-start) + '\n')
            distances[0].append(env.sim.data.qpos[0]-start)
    
print('pCN done, '+ str(time.ctime()))

In [ ]:
"""

TEST 1 (pCNL) - check if policy learned behaviour 

"""  


""" initialise NNs - pCNL """

method = 'pCNL'
N_data = 100

model_list = []

# Hyperparameters for our network
input_size = dim
n_layer = 3
hidden_sizes = [10,10,10]
output_size = 1

# Store hyperparameters in string
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])

for it in range(1000):
    # Build a feed-forward network
    model_list.append(nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[-1], output_size)))

    # Get weights
    weights = []
    biases = []
    for l in range(n_layer+1):
        weights.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_w'+str(l)+'_sampleNo'+str(it)+'.npy'))
        biases.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_b'+str(l)+'_sampleNo'+str(it)+'.npy'))
        
    # Set all weights to 0
    for l in range(n_layer+1):
        model_list[it][2*l].weight = torch.nn.Parameter(torch.from_numpy(weights[l]).float(), requires_grad=False)
        model_list[it][2*l].bias = torch.nn.Parameter(torch.from_numpy(biases[l]).float(), requires_grad=False)

        
""" run tests - pCNL """ 
        
v_with_noise = np.zeros(N_a)

print('Test started: ' + str(time.ctime()))
with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env.unwrapped
    
    for _ in range(10):
        for N_data in [100]:
            obs = env.reset()
            errors = 0  # to keep track of how many moves would be the same if using true policy
            corrects = 0

            ''' Store values taken to reproduce behaviour '''
            poss = []
            vels = []
            for k in range(100):
                progressBar(k+1,100)

                poss.append(obs[0:8])
                vels.append(obs[8:17])
                poss[k] = np.insert(poss[k],0, env.sim.data.qpos[0] )

                ''' See where different actions would take us, evaluate mean of value function samples there '''
                v = np.zeros(N_a)
                for it in range(1000): # Iterate through different neural networks
                    for i in range(N_a):
                        x = env.step(actions[i])[0]
                        v[i] += u(x).detach().numpy()[0]
                        env.set_state(poss[k], vels[k])
                v = v/1000 # to get mean of value function evaluations

                ''' Pick action which maximises the mean value function (plus noise) at the new location '''
                v_with_noise = v+sigma*np.random.normal(np.zeros(N_a),np.ones(N_a)) 
                a_argmax_with_noise = np.argmax(v_with_noise)
                
                ''' Check if action was optimal according to true policy; move according to learned behaviour  '''
                if a_argmax_with_noise != discrete_action_no_noise(policy(obs))[0]:
                    errors+=1
                else:
                    corrects+=1
                obs = env.step(actions[a_argmax_with_noise])[0] 

            print('\nErrorrate = ',(errors/(errors+corrects)))
#             input("Press Enter to start visualisation...")

            ''' Visualise the cheetah '''
            env.set_state(poss[0], vels[0])
            start = env.sim.data.qpos[0]       
            for k in range(100):
#                 env.render()
                env.set_state(poss[k], vels[k])

            print('Distance covered by Cheetah = ' + str(env.sim.data.qpos[0]-start) + '\n')   
            distances[1].append(env.sim.data.qpos[0]-start)
            
print('pCNL done, '+ str(time.ctime()))
np.save('BNN_dist.npy',distances)

In [ ]:
"""

TEST 2 - check how often mean of posterior function predicts the correct value

"""

""" initialise NNs - pCN """

method = 'pCN'
N_data = 100

model_list = []

# Hyperparameters for our network
input_size = dim
n_layer = 3
hidden_sizes = [10,10,10]
output_size = 1

# Store hyperparameters in string
hyps = str(hidden_sizes[0])
for i in range(1,n_layer):
    hyps = hyps+'_'+str(hidden_sizes[i])

for it in range(1000):
    # Build a feed-forward network
    model_list.append(nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                          nn.Tanh(),
                          nn.Linear(hidden_sizes[-1], output_size)))

    # Get weights
    weights = []
    biases = []
    for l in range(n_layer+1):
        weights.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_w'+str(l)+'_sampleNo'+str(it)+'.npy'))
        biases.append(np.load('np_saved/HC/samples_policy_learning/NN_'+hyps+'_'+method+'_NData'+str(N_data)+'_b'+str(l)+'_sampleNo'+str(it)+'.npy'))
        
    # Set all weights to 0
    for l in range(n_layer+1):
        model_list[it][2*l].weight = torch.nn.Parameter(torch.from_numpy(weights[l]).float(), requires_grad=False)
        model_list[it][2*l].bias = torch.nn.Parameter(torch.from_numpy(biases[l]).float(), requires_grad=False)


""" run tests """

x_test_100 = np.load('HC_x_test_100.npy')
a_test_100 = np.load('HC_a_test_100.npy')
optimal_choice = 0
not_optimal_choice = 0

start = time.time()
for j in range(100):
    v = np.zeros(N_a)
    
    for it in range(1000):
        for i in range(N_a):
            v[i] += u(x_test_100[i,:,j]).detach().numpy()[0]
            
    if a_test_100[j]==np.argmax(v):
        optimal_choice += 1
    else:
        not_optimal_choice += 1
            
print('Correct choices: ',optimal_choice/(optimal_choice+not_optimal_choice))
print('Wrong choices: ',not_optimal_choice/(optimal_choice+not_optimal_choice))
print('Time: ',time.time()-start)
print(str(time.ctime()))

In [ ]:
with tf.Session():
    tf_util.initialize()   
    env = gym.make('HalfCheetah-v2')
    env.unwrapped
    
    for _ in range(10):
#         input("Press Enter to start visualisation...")
        obs = env.reset()
        start = env.sim.data.qpos[0]    
        for k in range(100):
#             env.render()
            obs = env.step(discrete_action_no_noise(policy(obs))[1])[0] 
        print('Distance covered by Cheetah = ' + str(env.sim.data.qpos[0]-start) + '\n')   